# Laboratorio 4 - Modelado multidimensional y ETL inicial
### Inteligencia de Negocios
### Andrés Martin Ochoa Toro

WWI (World Wide Importers) es uan empresa encargada de realizar importaciones y venderlas a diferentes clientes en diferentes ciudades de Estados Unidos. En esta ocasión se desea optimizar las ganancias al analizar las ganancias que se están obteniendo con cada uno de los productos y mejorar aquellas que aún tengan un margen de ganancias por mejorar.

En este proceso, se plantea desde la construcción de la base de datos, la carga de los datos y la implementación de tres diferentes procesos de ETL.

# Importar librerías

In [858]:
import pandas as pd
import numpy as np

# Carga de los diferentes conjuntos de datos

In [859]:
data_city = pd.read_csv('./Datos/Completos/dimension_city.csv')
data_customer = pd.read_csv('./Datos/Completos/dimension_customer.csv')
data_date = pd.read_csv('./Datos/Completos/dimension_date.csv')
data_employee = pd.read_csv('./Datos/Completos/dimension_employee.csv')
data_stock_item = pd.read_csv('./Datos/Completos/dimension_stock_item.csv')
data_fact_order = pd.read_csv('./Datos/Completos/fact_order.csv')

# Análisis de los conjuntos de datos

In [860]:
print("Data City", data_city.shape)
print("Data Customer", data_customer.shape)
print("Data Date", data_date.shape)
print("Data Employee", data_employee.shape)
print("Data Stock Item", data_stock_item.shape)
print("Data Fact Order", data_fact_order.shape)

Data City (116295, 10)
Data Customer (403, 7)
Data Date (1461, 9)
Data Employee (213, 4)
Data Stock Item (672, 14)
Data Fact Order (231412, 15)


#  Dimension City

In [861]:
data_city.drop(["row ID"], axis=1, inplace=True)

In [862]:
print("Data City")
data_city.head(10)

Data City


,City_Key,City,State_Province,Country,Continent,Sales_Territory,Region,Subregion,Latest_Recorded_Population
0,0,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1,Carrollton,New York,United States,North America,Mideast,Americas,Northern America,0
2,2,Carrollton,Virginia,United States,North America,Southeast,Americas,Northern America,4574
3,3,Carrollton,Illinois,United States,North America,Great Lakes,Americas,Northern America,2484
4,4,Carrollton,Missouri,United States,North America,Plains,Americas,Northern America,3784
5,5,Carrollton,Ohio,United States,North America,Great Lakes,Americas,Northern America,3241
6,6,Carrollton,Kentucky,United States,North America,Southeast,Americas,Northern America,3938
7,7,Carrollton,Georgia,United States,North America,Southeast,Americas,Northern America,24388
8,8,Carrollton,Alabama,United States,North America,Southeast,Americas,Northern America,1019
9,9,Carrollton,Mississippi,United States,North America,Southeast,Americas,Northern America,190


In [863]:
data_city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116295 entries, 0 to 116294
Data columns (total 9 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   City_Key                    116295 non-null  int64 
 1   City                        116295 non-null  object
 2   State_Province              116294 non-null  object
 3   Country                     116294 non-null  object
 4   Continent                   116294 non-null  object
 5   Sales_Territory             116294 non-null  object
 6   Region                      116294 non-null  object
 7   Subregion                   116294 non-null  object
 8   Latest_Recorded_Population  116295 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 8.0+ MB


In [864]:
for col in data_city.columns:
    nan = data_city[col].isna().sum()
    print("La columna %s tiene %i valores nulos" % (col, nan))

La columna City_Key tiene 0 valores nulos
La columna City tiene 0 valores nulos
La columna State_Province tiene 1 valores nulos
La columna Country tiene 1 valores nulos
La columna Continent tiene 1 valores nulos
La columna Sales_Territory tiene 1 valores nulos
La columna Region tiene 1 valores nulos
La columna Subregion tiene 1 valores nulos
La columna Latest_Recorded_Population tiene 0 valores nulos


In [865]:
data_city_null = data_city.isnull().any(axis=1)

In [866]:
data_city_deleted_cols = data_city[data_city_null]
data_city_deleted_cols

,City_Key,City,State_Province,Country,Continent,Sales_Territory,Region,Subregion,Latest_Recorded_Population
0,0,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0


# Eliminar filas con valores nulos

Se procede a eliminar las filas en la dimensión City. En este caso se deciden eliminar los datos debido a que sólo es una dato.

In [867]:
data_city = data_city.dropna()
data_city

,City_Key,City,State_Province,Country,Continent,Sales_Territory,Region,Subregion,Latest_Recorded_Population
1,1,Carrollton,New York,United States,North America,Mideast,Americas,Northern America,0
2,2,Carrollton,Virginia,United States,North America,Southeast,Americas,Northern America,4574
3,3,Carrollton,Illinois,United States,North America,Great Lakes,Americas,Northern America,2484
4,4,Carrollton,Missouri,United States,North America,Plains,Americas,Northern America,3784
5,5,Carrollton,Ohio,United States,North America,Great Lakes,Americas,Northern America,3241
...,...,...,...,...,...,...,...,...,...
116290,116290,Keyser,West Virginia,United States,North America,Southeast,Americas,Northern America,5439
116291,116291,Keystone,West Virginia,United States,North America,Southeast,Americas,Northern America,282
116292,116292,Kiahsville,West Virginia,United States,North America,Southeast,Americas,Northern America,0
116293,116293,Kimball,West Virginia,United States,North America,Southeast,Americas,Northern America,194


Finalmente, se procede a guardar los datos resultantes después de procesar los valores nulos.

In [868]:
data_city.to_csv("./Datos Procesados/dimension_city.csv", sep=",", encoding="utf-8", index= False)

In [869]:
data_city_deleted_cols.to_csv("./Datos Eliminados/dimension_city_deleted.csv", sep=",", encoding="utf-8", index= False)

# Dimension Customer

In [870]:
print("Data Customer")
data_customer.head(10)

Data Customer


,Customer_Key,Customer,Bill_To_Customer,Category,Buying_Group,Primary_Contact,Postal_Code
0,0,Unknown,NaN,NaN,NaN,NaN,NaN
1,1,Tailspin Toys (Head Office),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Waldemar Fisar,90410.0
2,2,Tailspin Toys (Sylvanite- MT),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Lorena Cindric,90216.0
3,3,Tailspin Toys (Peeples Valley- AZ),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Bhaargav Rambhatla,90205.0
4,4,Tailspin Toys (Medicine Lodge- KS),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Daniel Roman,90152.0
5,5,Tailspin Toys (Gasport- NY),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Johanna Huiting,90261.0
6,6,Tailspin Toys (Jessie- ND),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Biswajeet Thakur,90298.0
7,7,Tailspin Toys (Frankewing- TN),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Kalidas Nadar,90761.0
8,8,Tailspin Toys (Bow Mar- CO),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Kanti Kotadia,90484.0
9,9,Tailspin Toys (Netcong- NJ),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Sointu Aalto,90129.0


In [871]:
data_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403 entries, 0 to 402
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Customer_Key      403 non-null    int64  
 1   Customer          403 non-null    object 
 2   Bill_To_Customer  402 non-null    object 
 3   Category          402 non-null    object 
 4   Buying_Group      402 non-null    object 
 5   Primary_Contact   402 non-null    object 
 6   Postal_Code       402 non-null    float64
dtypes: float64(1), int64(1), object(5)
memory usage: 22.2+ KB


In [872]:
for col in data_customer.columns:
    nan = data_customer[col].isna().sum()
    print("La columna %s tiene %i valores nulos" % (col, nan))

La columna Customer_Key tiene 0 valores nulos
La columna Customer tiene 0 valores nulos
La columna Bill_To_Customer tiene 1 valores nulos
La columna Category tiene 1 valores nulos
La columna Buying_Group tiene 1 valores nulos
La columna Primary_Contact tiene 1 valores nulos
La columna Postal_Code tiene 1 valores nulos


In [873]:
data_customer_null = data_customer.isnull().any(axis=1)

In [874]:
data_customer_deleted_cols = data_customer[data_customer_null]
data_customer_deleted_cols

,Customer_Key,Customer,Bill_To_Customer,Category,Buying_Group,Primary_Contact,Postal_Code
0,0,Unknown,NaN,NaN,NaN,NaN,NaN


# Eliminar filas con valores nulos

Se procede a eliminar las filas en la dimensión Customer. En este caso se deciden eliminar los datos debido a que sólo es una dato.

In [875]:
data_customer = data_customer.dropna()
data_customer

,Customer_Key,Customer,Bill_To_Customer,Category,Buying_Group,Primary_Contact,Postal_Code
1,1,Tailspin Toys (Head Office),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Waldemar Fisar,90410.0
2,2,Tailspin Toys (Sylvanite- MT),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Lorena Cindric,90216.0
3,3,Tailspin Toys (Peeples Valley- AZ),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Bhaargav Rambhatla,90205.0
4,4,Tailspin Toys (Medicine Lodge- KS),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Daniel Roman,90152.0
5,5,Tailspin Toys (Gasport- NY),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Johanna Huiting,90261.0
...,...,...,...,...,...,...,...
398,398,Wingtip Toys (Haycock- AK),Wingtip Toys (Head Office),Novelty Shop,Wingtip Toys,Bhaamini Kanaparthi,90603.0
399,399,Wingtip Toys (Kapa'a- HI),Wingtip Toys (Head Office),Novelty Shop,Wingtip Toys,Margit Molnar,90796.0
400,400,Wingtip Toys (Dickworsham- TX),Wingtip Toys (Head Office),Novelty Shop,Wingtip Toys,Chandni Jandhyala,90704.0
401,401,Wingtip Toys (Caro- MI),Wingtip Toys (Head Office),Novelty Shop,Wingtip Toys,Ljubomir Nesic,90043.0


In [876]:
data_customer.Postal_Code = data_customer.Postal_Code.astype(int)

Finalmente, se procede a guardar los datos resultantes después de procesar los valores nulos.

In [877]:
data_customer.to_csv("./Datos Procesados/dimension_customer.csv", sep=",", encoding="utf-8", index= False)

In [878]:
data_customer_deleted_cols.to_csv("./Datos Eliminados/dimension_customer_deleted.csv", sep=",", encoding="utf-8", index= False)

# Dimension Date

In [879]:
print("Data Date")
data_date.head(10)

Data Date


,Date_key,Day_Number,Day_val,Month_val,Short_Month,Calendar_Month_Number,Calendar_Year,Fiscal_Month_Number,Fiscal_Year
0,2013-01-01,1,1,January,Jan,1,2013,3,2013
1,2013-01-02,2,2,January,Jan,1,2013,3,2013
2,2013-01-03,3,3,January,Jan,1,2013,3,2013
3,2013-01-04,4,4,January,Jan,1,2013,3,2013
4,2013-01-05,5,5,January,Jan,1,2013,3,2013
5,2013-01-06,6,6,January,Jan,1,2013,3,2013
6,2013-01-07,7,7,January,Jan,1,2013,3,2013
7,2013-01-08,8,8,January,Jan,1,2013,3,2013
8,2013-01-09,9,9,January,Jan,1,2013,3,2013
9,2013-01-10,10,10,January,Jan,1,2013,3,2013


In [880]:
data_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Date_key               1461 non-null   object
 1   Day_Number             1461 non-null   int64 
 2   Day_val                1461 non-null   int64 
 3   Month_val              1461 non-null   object
 4   Short_Month            1461 non-null   object
 5   Calendar_Month_Number  1461 non-null   int64 
 6   Calendar_Year          1461 non-null   int64 
 7   Fiscal_Month_Number    1461 non-null   int64 
 8   Fiscal_Year            1461 non-null   int64 
dtypes: int64(6), object(3)
memory usage: 102.9+ KB


In [881]:
for col in data_date.columns:
    nan = data_date[col].isna().sum()
    print("La columna %s tiene %i valores nulos" % (col, nan))

La columna Date_key tiene 0 valores nulos
La columna Day_Number tiene 0 valores nulos
La columna Day_val tiene 0 valores nulos
La columna Month_val tiene 0 valores nulos
La columna Short_Month tiene 0 valores nulos
La columna Calendar_Month_Number tiene 0 valores nulos
La columna Calendar_Year tiene 0 valores nulos
La columna Fiscal_Month_Number tiene 0 valores nulos
La columna Fiscal_Year tiene 0 valores nulos


In [882]:
data_date_null = data_date.isnull().any(axis=1)

In [883]:
data_date_deleted_cols = data_date[data_date_null]
data_date_deleted_cols

,Date_key,Day_Number,Day_val,Month_val,Short_Month,Calendar_Month_Number,Calendar_Year,Fiscal_Month_Number,Fiscal_Year


# Dimension Employee

In [884]:
print("Data Employee")
data_employee.head(10)

Data Employee


,Employee_Key,Employee,Preferred_Name,Is_Salesperson
0,0,Unknown,NaN,False
1,1,Lily Code,Lily,True
2,2,Isabella Rupp,Isabella,False
3,3,Ethan Onslow,Ethan,False
4,4,Amy Trefl,Amy,True
5,5,Jai Shand,Jai,False
6,6,Anthony Grosse,Anthony,True
7,7,Taj Shand,Taj,True
8,8,Hudson Hollinworth,Hudson,True
9,9,Jack Potter,Jack,True


In [885]:
data_employee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Employee_Key    213 non-null    int64 
 1   Employee        213 non-null    object
 2   Preferred_Name  212 non-null    object
 3   Is_Salesperson  213 non-null    bool  
dtypes: bool(1), int64(1), object(2)
memory usage: 5.3+ KB


In [886]:
for col in data_employee.columns:
    nan = data_employee[col].isna().sum()
    print("La columna %s tiene %i valores nulos" % (col, nan))

La columna Employee_Key tiene 0 valores nulos
La columna Employee tiene 0 valores nulos
La columna Preferred_Name tiene 1 valores nulos
La columna Is_Salesperson tiene 0 valores nulos


In [887]:
data_employee_null = data_employee.isnull().any(axis=1)

In [888]:
data_employee_deleted_cols = data_employee[data_employee_null]
data_employee_deleted_cols

,Employee_Key,Employee,Preferred_Name,Is_Salesperson
0,0,Unknown,NaN,False


# Eliminar filas con valores nulos

Se procede a eliminar las filas en la dimensión Employee. En este caso se deciden eliminar los datos debido a que sólo es una dato.

In [889]:
data_employee = data_employee.dropna()
data_employee

,Employee_Key,Employee,Preferred_Name,Is_Salesperson
1,1,Lily Code,Lily,True
2,2,Isabella Rupp,Isabella,False
3,3,Ethan Onslow,Ethan,False
4,4,Amy Trefl,Amy,True
5,5,Jai Shand,Jai,False
...,...,...,...,...
208,208,Archer Lamble,Archer,True
209,209,Piper Koch,Piper,False
210,210,Katie Darwin,Katie,False
211,211,Jai Shand,Jai,False


Finalmente, se procede a guardar los datos resultantes después de procesar los valores nulos.

In [890]:
data_employee.to_csv("./Datos Procesados/dimension_employee.csv", sep=",", encoding="utf-8", index= False)

In [891]:
data_employee_deleted_cols.to_csv("./Datos Eliminados/dimension_employee_deleted.csv", sep=",", encoding="utf-8", index= False)

# Dimension Stock Item

In [892]:
print("Data Stock Item")
data_stock_item.head(10)

Data Stock Item


,Stock_Item_Key,Stock_Item,Color,Selling_Package,Buying_Package,Brand,Size_val,Lead_Time_Days,Quantity_Per_Outer,Is_Chiller_Stock,Tax_Rate,Unit_Price,Recommended_Retail_Price,Typical_Weight_Per_Unit
0,0,Unknown,NaN,NaN,NaN,NaN,NaN,0,0,False,",000",",00",",00",",000"
1,1,Void fill 400 L bag (White) 400L,NaN,Each,Each,NaN,400L,14,10,False,"14,000","50,00","74,75","1,000"
2,2,Void fill 300 L bag (White) 300L,NaN,Each,Each,NaN,300L,14,10,False,"14,000","37,50","56,06",",750"
3,3,Void fill 200 L bag (White) 200L,NaN,Each,Each,NaN,200L,14,10,False,"14,000","25,00","37,38",",500"
4,4,Void fill 100 L bag (White) 100L,NaN,Each,Each,NaN,100L,14,10,False,"14,000","12,50","18,69",",250"
5,5,Air cushion machine (Blue),NaN,Each,Each,NaN,NaN,20,1,False,"20,000","1899,00","2839,01","10,000"
6,6,Air cushion film 200mmx200mm 325m,NaN,Each,Each,NaN,325m,14,1,False,"14,000","90,00","134,55","6,000"
7,7,Air cushion film 200mmx100mm 325m,NaN,Each,Each,NaN,325m,14,1,False,"14,000","87,00","130,07","5,000"
8,8,Large replacement blades 18mm,NaN,Each,Each,NaN,18mm,14,10,False,"14,000","4,30","6,43",",800"
9,9,Small 9mm replacement blades 9mm,NaN,Each,Each,NaN,9mm,14,10,False,"14,000","4,10","6,13",",700"


In [893]:
data_stock_item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Stock_Item_Key            672 non-null    int64 
 1   Stock_Item                672 non-null    object
 2   Color                     360 non-null    object
 3   Selling_Package           671 non-null    object
 4   Buying_Package            671 non-null    object
 5   Brand                     67 non-null     object
 6   Size_val                  468 non-null    object
 7   Lead_Time_Days            672 non-null    int64 
 8   Quantity_Per_Outer        672 non-null    int64 
 9   Is_Chiller_Stock          672 non-null    bool  
 10  Tax_Rate                  672 non-null    object
 11  Unit_Price                672 non-null    object
 12  Recommended_Retail_Price  672 non-null    object
 13  Typical_Weight_Per_Unit   672 non-null    object
dtypes: bool(1), int64(3), obje

In [894]:
for col in data_stock_item.columns:
    nan = data_stock_item[col].isna().sum()
    print("La columna %s tiene %i valores nulos" % (col, nan))

La columna Stock_Item_Key tiene 0 valores nulos
La columna Stock_Item tiene 0 valores nulos
La columna Color tiene 312 valores nulos
La columna Selling_Package tiene 1 valores nulos
La columna Buying_Package tiene 1 valores nulos
La columna Brand tiene 605 valores nulos
La columna Size_val tiene 204 valores nulos
La columna Lead_Time_Days tiene 0 valores nulos
La columna Quantity_Per_Outer tiene 0 valores nulos
La columna Is_Chiller_Stock tiene 0 valores nulos
La columna Tax_Rate tiene 0 valores nulos
La columna Unit_Price tiene 0 valores nulos
La columna Recommended_Retail_Price tiene 0 valores nulos
La columna Typical_Weight_Per_Unit tiene 0 valores nulos


In [895]:
data_stock_item_null = data_stock_item.isnull().any(axis=1)

In [896]:
data_stock_item_null

0      True
1      True
2      True
3      True
4      True
       ... 
667    True
668    True
669    True
670    True
671    True
Length: 672, dtype: bool

In [897]:
data_stock_item_deleted_cols = data_stock_item[data_stock_item_null]
data_stock_item_deleted_cols

,Stock_Item_Key,Stock_Item,Color,Selling_Package,Buying_Package,Brand,Size_val,Lead_Time_Days,Quantity_Per_Outer,Is_Chiller_Stock,Tax_Rate,Unit_Price,Recommended_Retail_Price,Typical_Weight_Per_Unit
0,0,Unknown,NaN,NaN,NaN,NaN,NaN,0,0,False,",000",",00",",00",",000"
1,1,Void fill 400 L bag (White) 400L,NaN,Each,Each,NaN,400L,14,10,False,"14,000","50,00","74,75","1,000"
2,2,Void fill 300 L bag (White) 300L,NaN,Each,Each,NaN,300L,14,10,False,"14,000","37,50","56,06",",750"
3,3,Void fill 200 L bag (White) 200L,NaN,Each,Each,NaN,200L,14,10,False,"14,000","25,00","37,38",",500"
4,4,Void fill 100 L bag (White) 100L,NaN,Each,Each,NaN,100L,14,10,False,"14,000","12,50","18,69",",250"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,667,Air cushion machine (Blue),NaN,Each,Each,NaN,NaN,20,1,False,"20,000","1899,00","2839,01","10,000"
668,668,Void fill 100 L bag (White) 100L,NaN,Each,Each,NaN,100L,14,10,False,"14,000","12,50","18,69",",250"
669,669,Void fill 200 L bag (White) 200L,NaN,Each,Each,NaN,200L,14,10,False,"14,000","25,00","37,38",",500"
670,670,Void fill 300 L bag (White) 300L,NaN,Each,Each,NaN,300L,14,10,False,"14,000","37,50","56,06",",750"


Debido a la gran cantidad de valores nulos, no es conveniente borrar las filas. Esto representaria que se borraran alrededor de 605 datos de un total de 675. En este caso se decide procesar cada una de las columnas de manera independiente. 

En el caso de la columna color y size_val se decide reemplazar los valores nulos con un nuevo valor que represente que se desconoce el valor o no aplica. Y, en el caso de la columna Brand se decide eliminar debido a su gran número de valores nulos.

In [898]:
data_stock_item.drop(["Brand"], axis=1, inplace=True)

In [899]:
data_stock_item["Color"].fillna("Unknown", inplace=True)
data_stock_item["Size_val"].fillna("Unknown", inplace=True)

In [900]:
data_stock_item

,Stock_Item_Key,Stock_Item,Color,Selling_Package,Buying_Package,Size_val,Lead_Time_Days,Quantity_Per_Outer,Is_Chiller_Stock,Tax_Rate,Unit_Price,Recommended_Retail_Price,Typical_Weight_Per_Unit
0,0,Unknown,Unknown,NaN,NaN,Unknown,0,0,False,",000",",00",",00",",000"
1,1,Void fill 400 L bag (White) 400L,Unknown,Each,Each,400L,14,10,False,"14,000","50,00","74,75","1,000"
2,2,Void fill 300 L bag (White) 300L,Unknown,Each,Each,300L,14,10,False,"14,000","37,50","56,06",",750"
3,3,Void fill 200 L bag (White) 200L,Unknown,Each,Each,200L,14,10,False,"14,000","25,00","37,38",",500"
4,4,Void fill 100 L bag (White) 100L,Unknown,Each,Each,100L,14,10,False,"14,000","12,50","18,69",",250"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,667,Air cushion machine (Blue),Unknown,Each,Each,Unknown,20,1,False,"20,000","1899,00","2839,01","10,000"
668,668,Void fill 100 L bag (White) 100L,Unknown,Each,Each,100L,14,10,False,"14,000","12,50","18,69",",250"
669,669,Void fill 200 L bag (White) 200L,Unknown,Each,Each,200L,14,10,False,"14,000","25,00","37,38",",500"
670,670,Void fill 300 L bag (White) 300L,Unknown,Each,Each,300L,14,10,False,"14,000","37,50","56,06",",750"


In [901]:
data_stock_item['Tax_Rate'] = [x.replace(',','.') for x in data_stock_item['Tax_Rate']]
data_stock_item['Unit_Price'] = [x.replace(',','.') for x in data_stock_item['Unit_Price']]
data_stock_item['Recommended_Retail_Price'] = [x.replace(',','.') for x in data_stock_item['Recommended_Retail_Price']]
data_stock_item['Typical_Weight_Per_Unit'] = [x.replace(',','.') for x in data_stock_item['Typical_Weight_Per_Unit']]

In [902]:
data_stock_item

,Stock_Item_Key,Stock_Item,Color,Selling_Package,Buying_Package,Size_val,Lead_Time_Days,Quantity_Per_Outer,Is_Chiller_Stock,Tax_Rate,Unit_Price,Recommended_Retail_Price,Typical_Weight_Per_Unit
0,0,Unknown,Unknown,NaN,NaN,Unknown,0,0,False,.000,.00,.00,.000
1,1,Void fill 400 L bag (White) 400L,Unknown,Each,Each,400L,14,10,False,14.000,50.00,74.75,1.000
2,2,Void fill 300 L bag (White) 300L,Unknown,Each,Each,300L,14,10,False,14.000,37.50,56.06,.750
3,3,Void fill 200 L bag (White) 200L,Unknown,Each,Each,200L,14,10,False,14.000,25.00,37.38,.500
4,4,Void fill 100 L bag (White) 100L,Unknown,Each,Each,100L,14,10,False,14.000,12.50,18.69,.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,667,Air cushion machine (Blue),Unknown,Each,Each,Unknown,20,1,False,20.000,1899.00,2839.01,10.000
668,668,Void fill 100 L bag (White) 100L,Unknown,Each,Each,100L,14,10,False,14.000,12.50,18.69,.250
669,669,Void fill 200 L bag (White) 200L,Unknown,Each,Each,200L,14,10,False,14.000,25.00,37.38,.500
670,670,Void fill 300 L bag (White) 300L,Unknown,Each,Each,300L,14,10,False,14.000,37.50,56.06,.750


In [903]:
for col in data_stock_item.columns:
    nan = data_stock_item[col].isna().sum()
    print("La columna %s tiene %i valores nulos" % (col, nan))

La columna Stock_Item_Key tiene 0 valores nulos
La columna Stock_Item tiene 0 valores nulos
La columna Color tiene 0 valores nulos
La columna Selling_Package tiene 1 valores nulos
La columna Buying_Package tiene 1 valores nulos
La columna Size_val tiene 0 valores nulos
La columna Lead_Time_Days tiene 0 valores nulos
La columna Quantity_Per_Outer tiene 0 valores nulos
La columna Is_Chiller_Stock tiene 0 valores nulos
La columna Tax_Rate tiene 0 valores nulos
La columna Unit_Price tiene 0 valores nulos
La columna Recommended_Retail_Price tiene 0 valores nulos
La columna Typical_Weight_Per_Unit tiene 0 valores nulos


In [904]:
data_stock_item_null = data_stock_item.isnull().any(axis=1)

In [905]:
data_stock_item_null

0       True
1      False
2      False
3      False
4      False
       ...  
667    False
668    False
669    False
670    False
671    False
Length: 672, dtype: bool

In [906]:
data_stock_item_deleted_cols = data_stock_item[data_stock_item_null]
data_stock_item_deleted_cols

,Stock_Item_Key,Stock_Item,Color,Selling_Package,Buying_Package,Size_val,Lead_Time_Days,Quantity_Per_Outer,Is_Chiller_Stock,Tax_Rate,Unit_Price,Recommended_Retail_Price,Typical_Weight_Per_Unit
0,0,Unknown,Unknown,NaN,NaN,Unknown,0,0,False,.000,.00,.00,.000


# Eliminar filas con valores nulos

Se procede a eliminar las filas en la dimensión Employee. En este caso se deciden eliminar los datos debido a que sólo es una dato.

In [907]:
data_stock_item = data_stock_item.dropna()
data_stock_item

,Stock_Item_Key,Stock_Item,Color,Selling_Package,Buying_Package,Size_val,Lead_Time_Days,Quantity_Per_Outer,Is_Chiller_Stock,Tax_Rate,Unit_Price,Recommended_Retail_Price,Typical_Weight_Per_Unit
1,1,Void fill 400 L bag (White) 400L,Unknown,Each,Each,400L,14,10,False,14.000,50.00,74.75,1.000
2,2,Void fill 300 L bag (White) 300L,Unknown,Each,Each,300L,14,10,False,14.000,37.50,56.06,.750
3,3,Void fill 200 L bag (White) 200L,Unknown,Each,Each,200L,14,10,False,14.000,25.00,37.38,.500
4,4,Void fill 100 L bag (White) 100L,Unknown,Each,Each,100L,14,10,False,14.000,12.50,18.69,.250
5,5,Air cushion machine (Blue),Unknown,Each,Each,Unknown,20,1,False,20.000,1899.00,2839.01,10.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,667,Air cushion machine (Blue),Unknown,Each,Each,Unknown,20,1,False,20.000,1899.00,2839.01,10.000
668,668,Void fill 100 L bag (White) 100L,Unknown,Each,Each,100L,14,10,False,14.000,12.50,18.69,.250
669,669,Void fill 200 L bag (White) 200L,Unknown,Each,Each,200L,14,10,False,14.000,25.00,37.38,.500
670,670,Void fill 300 L bag (White) 300L,Unknown,Each,Each,300L,14,10,False,14.000,37.50,56.06,.750


Finalmente, se procede a guardar los datos resultantes después de procesar los valores nulos.

In [908]:
data_stock_item.to_csv("./Datos Procesados/dimension_stock_item.csv", sep=",", encoding="utf-8", index= False, decimal=",")

In [909]:
data_stock_item_deleted_cols.to_csv("./Datos Eliminados/dimension_stock_item_deleted.csv", sep=",", encoding="utf-8", index= False)

# Dimension Fact Order

In [910]:
print("Data Fact Order")
data_fact_order

Data Fact Order


,Order_Key,City_Key,Customer_Key,Stock_Item_Key,Order_Date_Key,Picked_Date_Key,Salesperson_Key,Picker_Key,Package,Quantity,Unit_Price,Tax_Rate,Total_Excluding_Tax,Tax_Amount,Total_Including_Tax
0,1,69644,0,153,2013-01-01,2013-01-01,15,0,Each,10,"230,00","15,000","2300,00","345,00","2645,00"
1,2,47692,0,170,2013-01-01,2013-01-01,6,0,Each,9,"13,00","15,000","117,00","17,55","134,55"
2,3,47692,0,210,2013-01-01,2013-01-01,6,0,Each,9,"32,00","15,000","288,00","43,20","331,20"
3,4,68504,105,106,2013-01-01,2013-01-01,4,0,Each,3,"30,00","15,000","90,00","13,50","103,50"
4,5,70644,57,14,2013-01-01,2013-01-01,19,11,Each,96,"2,70","15,000","259,20","38,88","298,08"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231407,231408,81497,150,122,2016-05-31,NaN,168,0,Each,48,"18,00","15,000","864,00","129,60","993,60"
231408,231409,109791,0,16,2016-05-31,NaN,172,0,Each,90,"32,00","15,000","2880,00","432,00","3312,00"
231409,231410,113642,218,143,2016-05-31,NaN,171,0,Each,72,"18,00","15,000","1296,00","194,40","1490,40"
231410,231411,104305,353,142,2016-05-31,NaN,19,0,Each,84,"18,00","15,000","1512,00","226,80","1738,80"


In [911]:
data_fact_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231412 entries, 0 to 231411
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   Order_Key            231412 non-null  int64 
 1   City_Key             231412 non-null  int64 
 2   Customer_Key         231412 non-null  int64 
 3   Stock_Item_Key       231412 non-null  int64 
 4   Order_Date_Key       231412 non-null  object
 5   Picked_Date_Key      228265 non-null  object
 6   Salesperson_Key      231412 non-null  int64 
 7   Picker_Key           231412 non-null  int64 
 8   Package              231412 non-null  object
 9   Quantity             231412 non-null  int64 
 10  Unit_Price           231412 non-null  object
 11  Tax_Rate             231412 non-null  object
 12  Total_Excluding_Tax  231412 non-null  object
 13  Tax_Amount           231412 non-null  object
 14  Total_Including_Tax  231412 non-null  object
dtypes: int64(7), object(8)
memory usag

In [912]:
for col in data_fact_order.columns:
    nan = data_fact_order[col].isna().sum()
    print("La columna %s tiene %i valores nulos" % (col, nan))

La columna Order_Key tiene 0 valores nulos
La columna City_Key tiene 0 valores nulos
La columna Customer_Key tiene 0 valores nulos
La columna Stock_Item_Key tiene 0 valores nulos
La columna Order_Date_Key tiene 0 valores nulos
La columna Picked_Date_Key tiene 3147 valores nulos
La columna Salesperson_Key tiene 0 valores nulos
La columna Picker_Key tiene 0 valores nulos
La columna Package tiene 0 valores nulos
La columna Quantity tiene 0 valores nulos
La columna Unit_Price tiene 0 valores nulos
La columna Tax_Rate tiene 0 valores nulos
La columna Total_Excluding_Tax tiene 0 valores nulos
La columna Tax_Amount tiene 0 valores nulos
La columna Total_Including_Tax tiene 0 valores nulos


In [913]:
data_fact_order_null = data_fact_order.isnull().any(axis=1)

In [914]:
data_fact_order_deleted_cols = data_fact_order[data_fact_order_null]
data_fact_order_deleted_cols

,Order_Key,City_Key,Customer_Key,Stock_Item_Key,Order_Date_Key,Picked_Date_Key,Salesperson_Key,Picker_Key,Package,Quantity,Unit_Price,Tax_Rate,Total_Excluding_Tax,Tax_Amount,Total_Including_Tax
2036,2037,47459,231,142,2013-01-12,NaN,27,0,Each,120,"18,00","15,000","2160,00","324,00","2484,00"
2590,2591,47005,197,134,2013-01-15,NaN,27,0,Each,108,"18,00","15,000","1944,00","291,60","2235,60"
2595,2596,66579,339,134,2013-01-15,NaN,15,0,Each,120,"18,00","15,000","2160,00","324,00","2484,00"
2597,2598,60508,0,134,2013-01-15,NaN,15,0,Each,120,"18,00","15,000","2160,00","324,00","2484,00"
3257,3258,71601,70,143,2013-01-19,NaN,28,0,Each,108,"18,00","15,000","1944,00","291,60","2235,60"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231407,231408,81497,150,122,2016-05-31,NaN,168,0,Each,48,"18,00","15,000","864,00","129,60","993,60"
231408,231409,109791,0,16,2016-05-31,NaN,172,0,Each,90,"32,00","15,000","2880,00","432,00","3312,00"
231409,231410,113642,218,143,2016-05-31,NaN,171,0,Each,72,"18,00","15,000","1296,00","194,40","1490,40"
231410,231411,104305,353,142,2016-05-31,NaN,19,0,Each,84,"18,00","15,000","1512,00","226,80","1738,80"


# Eliminar filas con valores nulos

Se procede a eliminar las filas en la dimensión Employee. En este caso se deciden eliminar los datos debido a que sólo es una dato.

In [915]:
data_fact_order = data_fact_order.dropna()
data_fact_order

,Order_Key,City_Key,Customer_Key,Stock_Item_Key,Order_Date_Key,Picked_Date_Key,Salesperson_Key,Picker_Key,Package,Quantity,Unit_Price,Tax_Rate,Total_Excluding_Tax,Tax_Amount,Total_Including_Tax
0,1,69644,0,153,2013-01-01,2013-01-01,15,0,Each,10,"230,00","15,000","2300,00","345,00","2645,00"
1,2,47692,0,170,2013-01-01,2013-01-01,6,0,Each,9,"13,00","15,000","117,00","17,55","134,55"
2,3,47692,0,210,2013-01-01,2013-01-01,6,0,Each,9,"32,00","15,000","288,00","43,20","331,20"
3,4,68504,105,106,2013-01-01,2013-01-01,4,0,Each,3,"30,00","15,000","90,00","13,50","103,50"
4,5,70644,57,14,2013-01-01,2013-01-01,19,11,Each,96,"2,70","15,000","259,20","38,88","298,08"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231397,231398,99099,205,161,2016-05-31,2016-05-31,168,142,Each,6,"25,00","15,000","150,00","22,50","172,50"
231398,231399,99099,205,218,2016-05-31,2016-05-31,168,142,Each,1,"25,00","15,000","25,00","3,75","28,75"
231399,231400,99099,205,118,2016-05-31,2016-05-31,168,142,Each,9,"35,00","15,000","315,00","47,25","362,25"
231400,231401,99099,205,50,2016-05-31,2016-05-31,168,142,Each,40,"102,00","15,000","4080,00","612,00","4692,00"


Finalmente, se procede a guardar los datos resultantes después de procesar los valores nulos.

In [916]:
data_fact_order.to_csv("./Datos Procesados/dimension_fact_order.csv", sep=",", encoding="utf-8", index= False)

In [917]:
data_stock_item_deleted_cols.to_csv("./Datos Eliminados/dimension_stock_item_deleted.csv", sep=",", encoding="utf-8", index= False)

Finalmente, en el data set se proceden a borrar todas aquellas filas que estén relacionadas con los datos eliminados en los otros data sets.

In [918]:
data_fact_order_city = ~data_fact_order.City_Key.isin(data_city_deleted_cols.City_Key)

In [919]:
data_fact_order_city.value_counts()

True    228265
Name: City_Key, dtype: int64

In [920]:
data_fact_order = data_fact_order[data_fact_order_city]

In [921]:
data_fact_order_customer = ~data_fact_order.Customer_Key.isin(data_customer_deleted_cols.Customer_Key)

In [922]:
data_fact_order_customer.value_counts()

True     143968
False     84297
Name: Customer_Key, dtype: int64

In [923]:
data_fact_order = data_fact_order[data_fact_order_customer]

In [924]:
data_fact_order_employee = ~data_fact_order.Salesperson_Key.isin(data_employee_deleted_cols.Employee_Key)

In [925]:
data_fact_order_employee.value_counts()

True    143968
Name: Salesperson_Key, dtype: int64

In [926]:
data_fact_order = data_fact_order[data_fact_order_employee]

In [927]:
data_fact_order_picker = ~data_fact_order.Picker_Key.isin(data_employee_deleted_cols.Employee_Key)

In [928]:
data_fact_order_picker.value_counts()

True     130901
False     13067
Name: Picker_Key, dtype: int64

In [929]:
data_fact_order = data_fact_order[data_fact_order_picker]

In [930]:
data_fact_order_stock_item = ~data_fact_order.Stock_Item_Key.isin(data_stock_item_deleted_cols.Stock_Item_Key)

In [931]:
data_fact_order_stock_item.value_counts()

True    130901
Name: Stock_Item_Key, dtype: int64

In [932]:
data_fact_order = data_fact_order[data_fact_order_stock_item]

In [933]:
data_fact_order_no_order_date_key = data_fact_order.Order_Date_Key.isin(data_date.Date_key)

In [934]:
data_fact_order_no_order_date_key.value_counts()

True    130901
Name: Order_Date_Key, dtype: int64

In [935]:
data_fact_order_no_pickerd_date_key = data_fact_order.Picked_Date_Key.isin(data_date.Date_key)

In [936]:
data_fact_order_no_pickerd_date_key.value_counts()

True    130901
Name: Picked_Date_Key, dtype: int64

In [937]:
data_fact_order

,Order_Key,City_Key,Customer_Key,Stock_Item_Key,Order_Date_Key,Picked_Date_Key,Salesperson_Key,Picker_Key,Package,Quantity,Unit_Price,Tax_Rate,Total_Excluding_Tax,Tax_Amount,Total_Including_Tax
4,5,70644,57,14,2013-01-01,2013-01-01,19,11,Each,96,"2,70","15,000","259,20","38,88","298,08"
5,6,70644,57,90,2013-01-01,2013-01-01,19,11,Each,5,"32,00","15,000","160,00","24,00","184,00"
6,7,70644,57,170,2013-01-01,2013-01-01,19,11,Each,2,"13,00","15,000","26,00","3,90","29,90"
37,38,69146,224,58,2013-01-01,2013-01-02,15,20,Each,100,"22,00","15,000","2200,00","330,00","2530,00"
38,39,69146,224,180,2013-01-01,2013-01-02,15,20,Each,10,"13,00","15,000","130,00","19,50","149,50"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231397,231398,99099,205,161,2016-05-31,2016-05-31,168,142,Each,6,"25,00","15,000","150,00","22,50","172,50"
231398,231399,99099,205,218,2016-05-31,2016-05-31,168,142,Each,1,"25,00","15,000","25,00","3,75","28,75"
231399,231400,99099,205,118,2016-05-31,2016-05-31,168,142,Each,9,"35,00","15,000","315,00","47,25","362,25"
231400,231401,99099,205,50,2016-05-31,2016-05-31,168,142,Each,40,"102,00","15,000","4080,00","612,00","4692,00"


In [938]:
data_fact_order['Unit_Price'] = [x.replace(',','.') for x in data_fact_order['Unit_Price']]
data_fact_order['Tax_Rate'] = [x.replace(',','.') for x in data_fact_order['Tax_Rate']]
data_fact_order['Total_Excluding_Tax'] = [x.replace(',','.') for x in data_fact_order['Total_Excluding_Tax']]
data_fact_order['Tax_Amount'] = [x.replace(',','.') for x in data_fact_order['Tax_Amount']]
data_fact_order['Total_Including_Tax'] = [x.replace(',','.') for x in data_fact_order['Total_Including_Tax']]

In [939]:
data_fact_order

,Order_Key,City_Key,Customer_Key,Stock_Item_Key,Order_Date_Key,Picked_Date_Key,Salesperson_Key,Picker_Key,Package,Quantity,Unit_Price,Tax_Rate,Total_Excluding_Tax,Tax_Amount,Total_Including_Tax
4,5,70644,57,14,2013-01-01,2013-01-01,19,11,Each,96,2.70,15.000,259.20,38.88,298.08
5,6,70644,57,90,2013-01-01,2013-01-01,19,11,Each,5,32.00,15.000,160.00,24.00,184.00
6,7,70644,57,170,2013-01-01,2013-01-01,19,11,Each,2,13.00,15.000,26.00,3.90,29.90
37,38,69146,224,58,2013-01-01,2013-01-02,15,20,Each,100,22.00,15.000,2200.00,330.00,2530.00
38,39,69146,224,180,2013-01-01,2013-01-02,15,20,Each,10,13.00,15.000,130.00,19.50,149.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231397,231398,99099,205,161,2016-05-31,2016-05-31,168,142,Each,6,25.00,15.000,150.00,22.50,172.50
231398,231399,99099,205,218,2016-05-31,2016-05-31,168,142,Each,1,25.00,15.000,25.00,3.75,28.75
231399,231400,99099,205,118,2016-05-31,2016-05-31,168,142,Each,9,35.00,15.000,315.00,47.25,362.25
231400,231401,99099,205,50,2016-05-31,2016-05-31,168,142,Each,40,102.00,15.000,4080.00,612.00,4692.00


In [940]:
data_fact_order.to_csv("./Datos Procesados/dimension_fact_order_final.csv", sep=",", encoding="utf-8", index= False)

In [942]:
data_date.to_csv("./Datos Procesados/dimension_date.csv", sep=",", encoding="utf-8", index= False)